In [48]:
from splinter import Browser

browser = Browser('chrome')
url = "http://www.google.com"

browser.visit(url)
search_input = browser.find_by_xpath('//*[@id="lst-ib"]')[0]
search_input.fill('facebook\n')


In [52]:
import time
url = 'https://www.facebook.com/luanyi.yen'
browser.visit(url)
browser.find_by_text('More').first.click()
browser.find_by_text('Likes').first.click()
# lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
a = 20
while a:
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(0.5)
    a -= 1

In [41]:
browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')